# Lopéz-Puertas et al. 2013

This notebook focusses on replicating the work of Lopéz-Puertas at al. (2013), where the researchers looked for PAHs making up the 3.3 μm emission recorded in VIMS data of Titan's upper atmosphere. This research used V1.20 of the NASA Ames PAH database for infrared spectroscopy and confirmed the presence of several PAHs

In [1]:
#import packages
import numpy as np
from amespahdbpythonsuite import observation
from amespahdbpythonsuite.amespahdb import AmesPAHdb
import matplotlib.pyplot as plt
from specutils import Spectrum1D


In [3]:
#read in the data file in ipac table format with the wavelength in microns and the flux in Jy
# data_file = "/Users/floorstikkelbroeck/Documents/Titan/titan_spec_900.ipac"
# data_file = "/Users/floorstikkelbroeck/Documents/Titan/titan_spec_950.ipac"
data_file = "/Users/floorstikkelbroeck/Documents/Titan/titan_spec_1000.ipac"
obs = observation.Observation(data_file)
obs.abscissaunitsto("1/cm") #convert the wavelength to wavenumbers

In [4]:
#load in the PAHdb with version 1.20 to mimic Lopéz-Puertas et al. 2013
# xml_file = "/Users/floorstikkelbroeck/Documents/Titan/PAHdb/pahdb-theoretical.xml"
xml_file = "/Users/floorstikkelbroeck/Documents/Titan/PAHdb/pahdb-theoretical-v1.20.xml"
pahdb = AmesPAHdb(
    filename=xml_file,
    check=False,
    cache=False,
)

                 AmesPAHdbPythonSuite
                 
                          by
                          
                Dr. Christiaan Boersma
                
                          and
                         
             Dr. Alexandros Maragkoudakis
             
               Dr. Matthew J. Shannanon
               
                  Dr. Joseph E. Roser
                 

          SUITE VERSION: 0.5.0.post62+g09a79ed         

                  CHECKING FOR UPDATE                  

                  NO UPDATE AVAILABLE                  

        WEBSITE: HTTP://WWW.ASTROCHEM.ORG/PAHDB/       

          CONTACT: CHRISTIAAN.BOERSMA@NASA.GOV         

     PARSING DATABASE: THIS MAY TAKE A FEW MINUTES     

==
FILENAME                    : /var/folders/vv/2nq91_0d0lsfm6gcskyz__r40000gn/T/a80b922979ad89624d56b35e6a3a1f28.pkl
PARSE TIME                  : 0:00:01.602711
VERSION (DATE)              : 1.20 (2011-01-13)
COMMENT                     : 

This is the NASA Ames

The database version of 1.20 contains 604 PAHs, including neutrals, cations and anions. We will not include:
 - Charged particals (not anticipated at that height, and low efficiencies)
 - species containing Magnesium, iron, silicon, oxygen

This will reduce the dataset to 202 PAHs

In [5]:
# Retrieve the transitions from the database for the subset of PAHs.
uids = pahdb.search("neutral mg=0 fe=0 si=0 o=0")
transitions = pahdb.gettransitionsbyuid(uids)

In [6]:
#Check the number of PAHs in the database
show = transitions.get()
show.keys()
len(show['uids']) #should be 202 PAHs, I am missing one??

201

In [7]:
# # Load the solar spectrum
# solar_spectrum_file = file_path / "resources/solar_spectrum.fits"
# with fits.open(solar_spectrum_file) as hdulist:
#     wavelength = hdulist[1].data["WAVELENGTH"]  # Assuming the wavelength is in nm
#     flux = hdulist[1].data["FLUX"]  # Assuming the flux is in units of irradiance

#     # Convert wavelength from nm to cm^-1 (wavenumber)
#     frequency = 1e7 / wavelength

#     # Prepare the solar spectrum dictionary
#     solar_spectrum = {
#         "frequency": np.flip(frequency),
#         "intensity": np.flip(flux)
#     }

# # Calculate the emission spectrum convolved with the solar spectrum.
# transitions.cascade(
#     solar_spectrum,
#     star=True,
#     stellar_model=True,
#     convolved=True,
#     multiprocessing=False,
#     cache=False,
# )

In [8]:
from SORCE import collect_irradiance_data
irradiance_data = collect_irradiance_data()
wavel = np.array(irradiance_data['august']['wavelengths'])
irrad = np.array(irradiance_data['august']['irradiances'])

[2.0154789248652157e-07, 2.557568176568268e-07, 8.62760361051425e-07, 3.448306972162778e-07, 5.571801683221394e-07]
[2.0711762501035122e-07, 2.649133110472845e-07, 8.63708099249916e-07, 3.4562316674003335e-07, 5.600750301820511e-07]
945


In [9]:
#I have a spectrum for UV photons, how do i cascade this with the PAHdb
# Assuming you have a UV spectrum dictionary similar to the solar_spectrum dictionary
uv_spectrum = {
    "frequency": 1e8 / np.flip(wavel),  # Replace with your UV spectrum frequencies
    "intensity": 1e3 *np.flip(irrad * wavel**2) / (4 * np.pi)   # Replace with your UV spectrum intensities
}

# Cascade with the UV spectrum
transitions.cascade(uv_spectrum, star=True, stellar_model=True, convolved=True, multiprocessing=True)


            APPLYING CASCADE EMISSION MODEL            

 STELLAR MODEL SELECTED: USING FIRST PARAMETER AS MODEL

          REBINNING STELLAR MODEL: 100 POINTS          

CALCULATED EFFECTIVE TEMPERATURE: 39076.335151714025 Kelvin

         CONVOLVING WITH ENTIRE RADIATION FIELD        

           USING MULTIPROCESSING WITH 7 CORES          



KeyboardInterrupt: 

In [ ]:
# Calculate the emission spectrum at the temperature reached
# after absorbing a 6 eV (CGS units) photon.
transitions.cascade(energy_UV * 1.603e-12, multiprocessing=True) #Figure this out for Lopez-Puertas et al. 2013

# Shift data 15 wavenumber to the red to mimic some effects of anharmonicity.
transitions.shift(-15.0)

In [ ]:
# Convolve the bands with a Gaussian with FWHM of 15 /cm.
spectrum = transitions.convolve(
    grid=obs.getgrid(), fwhm=15.0, gaussian=True, multiprocessing=False
)

In [ ]:
# Fit the spectrum
fit = spectrum.fit(obs)

In [ ]:
fit.plot(wavelength=True)
fit.plot(wavelength=True, residual=True)


In [ ]:
fit_info = fit.get()
fit_info.keys()
fit_info['weights']

Get the molecular structure of the UID of the most present molecules in the fit

In [ ]:
# Get the weights from fit_info
weights = fit_info['weights']

# Sort the uids by their weights in descending order
sorted_uids_by_weight = sorted(uids_fit, key=lambda uid: weights[uid], reverse=True)

# Print the uids and their formulas
for uid in sorted_uids_by_weight:
    species_uid = pahdb.getspeciesbyuid([uid]).get()
    mol_for = species_uid['data'][uid]['formula']
    print(f'The uid is: {uid}, the formula is: {mol_for}, the weight is: {weights[uid]}')


In [16]:
#Shows contibution of large vs small particles
# fit.plot(wavelength=True, size=True)

In [17]:
#shows contribution of neutrals or charged molecules (not applicable now because only neutrals considered)
# fit.plot(wavelength=True, charge=True)

In [ ]:
#contribution from pure or nitrogen containing PAHs
fit.plot(wavelength=True, composition=True)

In [ ]:
# Predict 3.15 - 3.40 µm spectrum
transitions.intersect(fit.getuids())
spectrum = transitions.convolve(gaussian=True, multiprocessing=False, grid=obs.getgrid())
coadded = spectrum.coadd(weights=fit.getweights())
coadded.plot()

In [20]:
#Get the fit spectrum so i can plot it in the next cell
spectrum_output = fit.observation
flux_values = spectrum_output.flux.value

In [21]:
#get the original observed spectrum
obs_output = obs.get()
original_obs = obs_output['spectrum'].flux.value
original_obs_x = obs_output['spectrum'].spectral_axis.value
original_obs_x = 1e4/original_obs_x
# help(obs)

In [ ]:
plt.plot(coadded.grid, coadded.data[0], label='Coadded Spectrum', color = 'purple')
# Plot the fit spectrum
plt.plot(fit.grid, flux_values, label='Fit Spectrum', color = 'orange')
plt.plot(original_obs_x, original_obs, label='Original Spectrum', color = 'blue')
#invert the x-axis to show the wavenumbers in increasing order
plt.gca().invert_xaxis()

# Add labels and legend
plt.xlabel('Wavelength (µm)')
plt.ylabel('Intensity')
plt.legend()
plt.title('Coadded and Fit Spectra')

# Show the plot
plt.show()

# Monte Carlo Fit

In [ ]:
# Fit the spectrum using Monte Carlo approach.
mcfit = spectrum.mcfit(obs, samples=1024, multiprocessing=False)

In [ ]:
# mcfit.plot(wavelength=True)
mcfit.plot(wavelength=True, residual=True)
mcfit.plot(wavelength=True, size=True)
mcfit.plot(wavelength=True, charge=True)
# mcfit.plot(wavelength=True, composition=True)
# mcfit.plot(wavelength=True, save=True, ftype="pdf")

In [ ]:
mcfit_info = mcfit.get()
mcfit_info

In [ ]:
mcfit.getfit()

In [ ]:
mcfit.getbreakdown()